<a href="https://colab.research.google.com/github/jarifNmohsin/CNN/blob/main/CNN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [28]:
# Convert MNIST Image into a Tensor of 4-Dimensions (no. of images, Height, Width, Color Channels)
transform = transforms.ToTensor()

In [29]:
# Train Data
train_data = datasets.MNIST(root='/cnn_data', train=True, download=True, transform=transform)

In [30]:
# Test Data
test_data = datasets.MNIST(root='/cnn_data', train=False, download=True, transform=transform)

In [31]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [32]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [33]:
# Create a small batch size for images
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [34]:
# Define our CNN Model
# Describe convolutional layer (two)
conv1 = nn.Conv2d(1, 6, 3, 1)
conv2 =  nn.Conv2d(6, 16, 3, 1)

In [35]:
# Grab one MNIST record/image
for i, (X_train, y_train) in enumerate(train_data):
  break

In [36]:
X_train.shape

torch.Size([1, 28, 28])

In [37]:
x = X_train.view(1, 1, 28, 28)

In [38]:
# Perform our first convulution
x = F.relu(conv1(x)) # Rectified Linear Unit for our activation function

In [39]:
# 1 single image, 6 is the filters we asked for, 26x26 (as we didn't put any padding)
x.shape

torch.Size([1, 6, 26, 26])

In [40]:
# pass thru the pooling layer
x = F.max_pool2d(x, 2, 2) # kernel of 2 and stride of 2

In [41]:
x.shape # 26 / 2 = 13

torch.Size([1, 6, 13, 13])

In [42]:
# Do our second convolutional layer
x = F.relu(conv2(x))

In [43]:
x.shape # Again we didn't put a padding so we lose 2 pixels around the outside of the image

torch.Size([1, 16, 11, 11])

In [44]:
# Pooling layer
x = F.max_pool2d(x, 2, 2)

In [45]:
x.shape # 11 / 2 = 5.5 but we have it round down, because we can't invent data to round up

torch.Size([1, 16, 5, 5])

In [46]:
((28-2) / 2 - 2) / 2

5.5

In [51]:
# Model Class
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 6, 3, 1)
    self.conv2 = nn.Conv2d(6, 16, 3, 1)
    # Fully connected layer
    self.fc1 = nn.Linear(5*5*16, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, X):
    X = F.relu(self.conv1(X))
    X = F.max_pool2d(X, 2, 2)
    # Second pass
    X = F.relu(self.conv2(X))
    X = F.max_pool2d(X, 2, 2)

    # Review to flatten it out
    X = X.view(-1, 16*5*5) # -1 because we can vary the batch size

    # Fully connected Layers
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.fc3 # No relu as it is the last layer
    return F.log_softmax(X, dim=1)


In [52]:
# Create an instance of our Model
torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [53]:
# Loss function optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)